In [2]:
from langchain_community.document_loaders import DocusaurusLoader


import nest_asyncio

nest_asyncio.apply()

loader = DocusaurusLoader("https://python.langchain.com")
docs = loader.load()
loader = DocusaurusLoader(
    "https://python.langchain.com",
    filter_urls=[
        "https://python.langchain.com/docs/integrations/document_loaders/sitemap"
    ],
)
documents = loader.load()
documents[0]

Fetching pages: 100%|###########################################################################################################| 1/1 [00:00<00:00,  9.22it/s]


Document(page_content='\n\n\n\n\nSitemap | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchProvidersProvidersAnthropicAWSGoogleHugging FaceMicrosoftOpenAIMoreComponentsChat modelsLLMsEmbedding modelsDocument loadersacreomAirbyteLoaderAirbyte CDK (Deprecated)Airbyte Gong (Deprecated)Airbyte Hubspot (Deprecated)Airbyte JSON (Deprecated)Airbyte Salesforce (Deprecated)Airbyte Shopify (Deprecated)Airbyte Stripe (Deprecated)Airbyte Typeform (Deprecated)Airbyte Zendesk Support (Deprecated)AirtableAlibaba Cloud MaxComputeAmazon TextractApify DatasetArcGISArxivAssemblyAI Audio TranscriptsAstraDBAsync ChromiumAsyncHtmlAthenaAWS S3 DirectoryAWS S3 FileAZLyricsAzure AI DataAzure Blob Storage ContainerAzure Blob Storage FileAzure AI Document IntelligenceBibTeXBiliBiliBlackboardBlockchainBrave 

In [3]:
import os
from langchain_community.embeddings import LlamaCppEmbeddings

llama_embedings = LlamaCppEmbeddings(model_path=os.path.expanduser("~/Downloads/Hermes-2-Pro-Mistral-7B.Q5_0.gguf"))

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /Users/ben/Downloads/Hermes-2-Pro-Mistral-7B.Q5_0.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_0     [  4096, 32032,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q5_0     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_0     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_0     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_0     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_0     [  

In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=llama_embedings)

In [ ]:
from langchain_google_vertexai import VertexAI

llm = VertexAI(model_name="gemini-pro")

In [ ]:
from langchain import hub

retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")